# **La bagarre**

---



## Preamble

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

### Global variables

In [ ]:
batch_size = 512
data_path = './data'
input_shape = (224,224,3)
num_classes = 6

### Imports

In [ ]:
import os
import numpy as np
import cv2
import PIL

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping 
from tensorflow.keras.applications import NASNetMobile
from sklearn.metrics import confusion_matrix, classification_report

import pandas as pd
import matplotlib.pyplot as plt
import inspect
from tqdm import tqdm

### Functions

In [ ]:
def random_images(n):
    images = []
    i = 1
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            images.append(os.path.join(dirname, filename))
    samples = np.random.randint(len(filenames), size=n)
    print(f'Displaying {n} images from a total of {len(images)} found.')
    f = plt.figure(figsize=(2*n,15))
    for sample in samples:
        sample_image = cv2.imread(images[sample])
        f.add_subplot(1, n, i)
        plt.imshow(sample_image)
        i = i + 1

## Data

In [ ]:
random_images(8)

In [ ]:
#!pip install split-folders
import splitfolders as sf

sf.ratio(f'{data_path}/raw',
  output=f'{data_path}/processed',
  seed=42,
  ratio=(.8, .1, .1),
  group_prefix=None,
  move=False)

In [ ]:
idg_aug = ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

idg_raw = ImageDataGenerator(rescale=1./255)


train = idg_aug.flow_from_directory(f'{data_path}/processed/train',
                                subset='training',
                                target_size=(224, 224),
                                class_mode="categorical",
                                batch_size=batch_size,
                                shuffle=False,
                                seed=42)

val = idg_raw.flow_from_directory(f'{data_path}/processed/val',
                                subset='training',
                                target_size=(224, 224),
                                class_mode="categorical",
                                shuffle=False,
                                seed=42)

test = idg_raw.flow_from_directory(f'{data_path}/processed/test',
                                subset='training',
                                target_size=(224, 224),
                                class_mode="categorical",
                                shuffle=False,
                                seed=42)

## Train Model

In [ ]:
# Reduce LR
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=0.00001)

# Checkpoints
checkpoint_filepath = "./model/checkpoint"
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath,
                             monitor='val_loss', verbose=1,
                             save_weights_only=True,
                             save_best_only=True, mode='min')

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=5,
                           restore_best_weights=True,
                           mode='min')

callbacks = [checkpoint, early_stop, reduce_lr]

In [ ]:
pre_trained_model = NASNetMobile(include_top=False, pooling='avg', input_shape=input_shape)
pre_trained_model.trainable = False
clf_model = tf.keras.models.Sequential()
clf_model.add(pre_trained_model)
clf_model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
clf_model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
#clf_model.load_weights(checkpoint_filepath)
history = clf_model.fit(train, callbacks=callbacks, epochs=20, validation_data=val)

In [ ]:
clf_model.save('./model/NASNetMobile-mednist')

## Evaluate Model

In [20]:
model = tf.keras.models.load_model('./model/NASNetMobile-mednist')
y_pred = model.predict(test)
y_pred = np.argmax(y_pred, axis=1)

In [21]:
print('Confusion Matrix')
print(confusion_matrix(test.classes, y_pred))

Confusion Matrix
[[800   0   0   0   0   0]
 [  0 717   0   0   0   0]
 [  0   0 800   0   0   0]
 [  0   0   0 800   0   0]
 [  0   0   0   0 800   0]
 [  0   0   0   0   0 802]]


In [22]:
print('Classification Report')
print(classification_report(test.classes, y_pred))

Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       800
           1       1.00      1.00      1.00       717
           2       1.00      1.00      1.00       800
           3       1.00      1.00      1.00       800
           4       1.00      1.00      1.00       800
           5       1.00      1.00      1.00       802

    accuracy                           1.00      4719
   macro avg       1.00      1.00      1.00      4719
weighted avg       1.00      1.00      1.00      4719

